### Model 1 - Enviormental sound classifiers

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import librosa
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.utils import to_categorical

from sklearn import metrics 
# Load metadata
metadata = pd.read_csv('C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/metadata/model1.csv')

In [3]:
# mfcc
def mfcc(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = 174 - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
 
    return mfccs


In [4]:
# cnn
def cnn(num_rows=40, num_columns=174, num_labels=10):
    model = Sequential()
    model.add(Conv2D(filters=16, kernel_size=2, input_shape=(num_rows, num_columns, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=32, kernel_size=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=64, kernel_size=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=128, kernel_size=2, activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(num_labels, activation='softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model

In [ ]:
fold_accuracies = []
for i in range(1, 11): # 10-fold
    print(f"Processing fold {i}")
    test_data = metadata[metadata['fold'] == i]
    train_data = metadata[metadata['fold'] != i]
    
    train_features = []
    test_features = []
    for idx, row in train_data.iterrows():
        train_features.append([mfcc(os.path.join('C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio',
                                                              'fold' + str(row['fold']), row['slice_file_name'])), row['classID']])
    for idx, row in test_data.iterrows():
        test_features.append([mfcc(os.path.join('C:/Users/zzzl0/Desktop/predicting-and-avoiding-dog-barking-behaviour/predicting-and-avoiding-dog-barking-behaviour/UrbanSound8K/audio',
                                                             'fold' + str(row['fold']), row['slice_file_name'])), row['classID']])
    train_features_df = pd.DataFrame(train_features, columns=['feature', 'class_label'])
    test_features_df = pd.DataFrame(test_features, columns=['feature', 'class_label'])
    
    X_train = np.array(train_features_df.feature.tolist())
    y_train = np.array(train_features_df.class_label.tolist())
    X_test = np.array(test_features_df.feature.tolist())
    y_test = np.array(test_features_df.class_label.tolist())
    
    le = LabelEncoder()
    y_train = to_categorical(le.fit_transform(y_train))
    y_test = to_categorical(le.fit_transform(y_test))
    
    X_train = X_train.reshape(X_train.shape[0], 40, 174, 1)
    X_test = X_test.reshape(X_test.shape[0], 40, 174, 1)
    
    model = cnn()
    
    model.fit(X_train, y_train, epochs=50, batch_size=256, verbose=1)
    
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(y_test, axis=1)
    fold_accuracy = accuracy_score(true_classes, predicted_classes)
    print(f"Accuracy for fold {i}: {fold_accuracy}")
    fold_accuracies.append(fold_accuracy)

print(f"10-fold cross validation accuracy: {np.mean(fold_accuracies)}")